## 字节级 BPE 训练算法：完整流程（纯文字说明）

本节给出从零实现字节级 BPE（Byte Pair Encoding）训练函数的详细步骤说明，涵盖输入输出约定、数据结构、预分词策略、主训练循环、边界与性能考量，以及与测试对齐的注意点。内容仅为算法与实现思路，不包含具体答案代码。

### 目标与输入/输出
- 输入
  - `input_path: str | Path`：UTF-8 文本语料路径。
  - `vocab_size: int`：最终词表容量上限（= 256 字节初始 + 合并产生的新 token + 特殊 token）。


#### PS：为什么这里提到“256”？
- 不是说 `vocab_size` 固定为 256；256 指“初始字节词表”的大小（8 位字节共有 2^8=256 种取值）。
- 你传入的 `vocab_size` 控制的是最终容量上限：`256（初始字节）+ merges（训练产生）+ special_tokens` 总数不得超过它。
- 如果 `vocab_size < 256 + len(special_tokens)`，训练在逻辑上不可行（连基础字节和特殊标记都放不下），应当报错或直接早停。
- 选择更大的 `vocab_size` 会带来更短的输入序列（更多合并），但也会增大嵌入矩阵和 softmax 的计算与显存开销，需要在压缩率与计算成本之间权衡。
- 使用 256 作为初始集合的动机：覆盖任何 UTF‑8 文本，无 OOV；若少于 256，将无法表达某些字节模式，导致不可逆或失败。


  - `special_tokens: list[str]`：要加入词表的特殊 token，它们不参与训练统计。
- 输出
  - `vocab: dict[int, bytes]`：token id -> 原始字节串。初始含 256 个单字节映射，随后追加合并产物与特殊 token。
  - `merges: list[tuple[bytes, bytes]]`：按顺序记录每次合并的左右字节串 `(left_bytes, right_bytes)`。

### 总体思路
1. 读入语料为字符串，构造“GPT-2 预分词正则”（需把 `special_tokens` 放在前面优先匹配）。
2. 用该正则切分出预分词字符串，统计其频次（`Counter`）。对特殊 token 本身，不计入频次（避免参与合并）。
3. 将每个预分词转成 UTF-8 字节序列，并存为 `tuple[int, ...]`（便于做字典键），得到 `word_freq: {token_tuple: count}`。
4. 初始化 `vocab`：包含 256 个字节 token（id 0..255），然后依次加入 `special_tokens` 的字节串（若未超出上限）。
5. 进入训练循环：
   - 统计所有 token 中相邻 pair 的出现次数（按 token 频次加权）。
   - 取最高频 pair，作为下一次合并目标；若不存在，提前结束。
   - 新 token 的字节串 = 左右子串字节拼接；记录到 `merges`；在 `vocab` 中分配新 id。
   - 将所有 token 序列中的该 pair 左到右、不重叠地替换为新 id，合并相同序列的频次。
   - 若词表达到 `vocab_size`，或无可合并对，则结束。
6. 返回 `(vocab, merges)`。

### 数据结构与表示
- 预分词输出：字符串序列（后续统一 `.encode('utf-8')`）。
- `word_freq: dict[tuple[int, ...], int]`：每个 token 序列（以字节 id 构成的元组）对应的频次。
- `pair_counts: Counter[tuple[int, int]]`：所有相邻对（如 `(97,98)`）的出现次数（累计各 token 的频次）。
- `vocab: dict[int, bytes]`：初始 `{i: bytes([i]) for i in range(256)}`，随后追加新 token 与特殊 token。
- `merges: list[tuple[bytes, bytes]]`：记录左、右子串的原始字节串（而非 id），便于重建 tokenizer。

### 预分词（GPT-2 正则 + 特殊 token 保护）
- 基础 GPT-2 正则模式（字符串）：
  - `'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+`
  - 需用第三方 `regex` 包（支持 Unicode 类别与负向前瞻）。
- 将 `special_tokens` 预先 `regex.escape` 后，以 `(?:tok1|tok2|...)|<GPT-2基础模式>` 的形式拼接，确保特殊 token 优先整体匹配。
- `pattern.findall(text)` 得到预分词列表；对在 `special_tokens` 集合内的分片，不计入 `word_freq` 统计。

### 初始化词表与 id 分配
- 起始：0..255 -> 单字节 `bytes([i])`。
- `next_token_id` 初始为 256。
- 依序将 `special_tokens` 的 UTF-8 字节串加入词表，遇到 `vocab_size` 上限则停止追加。
- 若此时 `next_token_id >= vocab_size`，可直接返回（无训练空间）。

### 主训练循环（核心）
1. 统计相邻 pair
   - 遍历 `word_freq` 中的每个 `token_tuple` 及其 `freq`。
   - 若 `len(token_tuple) >= 2`，将相邻对 `(t[i], t[i+1])` 的计数 `+= freq`。
   - 若 `pair_counts` 为空，提前结束。
2. 选频次最高的 pair
   - 取 `best_pair = argmax(pair_counts)`。
   - 将 `best_pair` 的左右 id 映射回 `bytes`：`left_bytes = vocab[left_id]`，`right_bytes = vocab[right_id]`。
   - `new_token_bytes = left_bytes + right_bytes`。
3. 记录 merge 与扩充词表
   - `merges.append((left_bytes, right_bytes))`。
   - `vocab[next_token_id] = new_token_bytes`。
4. 全量替换构建新 `word_freq`
   - 对每个旧的 `token_tuple`，进行“左到右、不重叠”的 pair 替换：
     - 遍历索引 `i`：若 `t[i], t[i+1] == best_pair` 则写入 `new_id` 并 `i += 2`，否则写入 `t[i]` 并 `i += 1`。
     - 替换完成得到新元组 `merged_tuple`，其频次累加到 `updated_word_freq[merged_tuple] += freq`。
   - 令 `word_freq = updated_word_freq`。
5. 终止判定
   - `next_token_id += 1`；若 `next_token_id >= vocab_size` 则结束。

### 结束条件
- 词表容量达到 `vocab_size`。
- 再无可合并 pair（所有 token 均为长度 1，或 pair 计数为空）。

### 性能与实现细节建议
- 在小语料（测试集提供）上，朴素实现就能达标（目标 < 1.5s）；注意避免多余复制。
- 关键点：
  - 使用 `Counter` 统计 pair，减少 Python 层逻辑。
  - `token_tuple` 用不可变 `tuple[int,...]` 作为 dict 键，更新时一次性构造。
  - 每轮只做一次全量替换与重建 `word_freq`。
- 若仍慢：检查是否有嵌套重复扫描、无谓的 `bytes` ↔ `tuple` 往返。

### 正确性检查要点
- 特殊 token 不应出现在 `merges` 与普通 vocab 值中（仅作为独立条目加入词表）。
- `merges` 的元素必须是“原始字节串对”，顺序与训练顺序一致。
- `vocab` 的键集合与参考集合一致、值集合与参考集合一致（测试是集合对比，而非顺序完全一致）。
- 替换必须是不重叠的左到右策略。

### 与测试对齐的注意点
- 使用 `regex` 包与给定 GPT-2 正则；普通 `re` 可能太慢或不支持。
- 预分词阶段需要把 `special_tokens` 放在正则前缀位置优先匹配，并从统计中剔除。
- `run_train_bpe` 测试会读取 `tests/fixtures` 下的小语料，比较 `merges` 与 `vocab` 的集合。
- 性能测试目标：在该小语料上 < 1.5 秒。

### 常见坑
- 忘记把 `special_tokens` 提前匹配或从统计中移除，导致 `merges` 出现 `<|` 等片段。
- 词表上限计算错误：记得包含 256 字节 + 新 token + 特殊 token。
- 替换逻辑重叠（未 `i += 2`），或顺序错误（非左到右）。
- 用字符串层面合并而非字节层面合并。

### 复杂度（朴素实现）
- 设每轮遍历所有 token 的总长度为 L，词频项数为 M，轮数为 R：
  - 计数相邻对：O(L)；
  - 全量替换：O(L)；
  - 总体：O(R·L)。
- 在测试语料上足以通过时间门槛。


## BPE 训练算法详细步骤分解

让我们从整体到细节，一步步分解整个 BPE 训练过程。

### 第一步：读取和预处理语料

#### 1.1 读取文件
```
输入: input_path (文件路径)
操作: 
  - 打开文件，使用 UTF-8 编码读取
  - 将整个文件内容读入内存作为一个字符串
输出: text (字符串)
```

#### 1.2 构建预分词正则表达式
```
输入: special_tokens (特殊标记列表)
操作:
  a. 定义基础 GPT-2 正则模式字符串:
     pattern_str = r"'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"
  
  b. 处理特殊标记:
     - 对每个 special_token 进行 regex.escape() 转义
     - 用 "|" 连接所有转义后的特殊标记
     - 如果有特殊标记，构造: "(?:" + 特殊标记模式 + ")|" + GPT-2基础模式
     - 如果没有特殊标记，直接使用 GPT-2 基础模式
  
  c. 编译正则表达式:
     pattern = regex.compile(最终模式字符串)

输出: pattern (编译后的正则对象)
```

#### 1.3 执行预分词
```
输入: text (原始文本), pattern (正则对象)
操作:
  - 使用 pattern.findall(text) 获取所有匹配的片段
  - 得到一个字符串列表，每个元素是一个预分词单元
输出: pre_tokens (字符串列表)

例子: "Hello world!" → ["Hello", " world", "!"]
```

### 第二步：统计预分词频次并转换为字节序列

#### 2.1 统计预分词频次
```
输入: pre_tokens (预分词列表), special_tokens (特殊标记集合)
操作:
  a. 创建一个 Counter 对象或空字典
  b. 遍历每个预分词:
     - 如果该预分词在 special_tokens 集合中，跳过（不统计）
     - 否则，将该预分词的计数 +1
  
输出: word_counts (字典: 字符串 → 出现次数)

例子: ["Hello", " world", "Hello", "!"] → {"Hello": 2, " world": 1, "!": 1}
```

#### 2.2 将字符串转换为字节序列
```
输入: word_counts (字符串频次字典)
操作:
  a. 创建新字典 word_freq = {}
  b. 对每个 (word, count) 对:
     - 将 word 转为字节: word_bytes = word.encode('utf-8')
     - 将字节序列转为整数元组: token_tuple = tuple(word_bytes)
       （每个字节值是 0-255 的整数）
     - 存储: word_freq[token_tuple] = count
  
输出: word_freq (字典: 字节元组 → 频次)

例子: 
  "Hi" → b'Hi' → (72, 105) 
  "你好" → b'\xe4\xbd\xa0\xe5\xa5\xbd' → (228, 189, 160, 229, 165, 189)
```

### 第三步：初始化词表

#### 3.1 创建基础字节词表
```
输入: 无
操作:
  a. 创建空字典 vocab = {}
  b. 对于 i 从 0 到 255:
     - vocab[i] = bytes([i])
     （每个整数 ID 映射到对应的单字节）
  
输出: vocab (包含 256 个初始映射)

例子:
  vocab[65] = b'A'
  vocab[97] = b'a'
  vocab[255] = b'\xff'
```

#### 3.2 添加特殊标记到词表
```
输入: vocab, special_tokens, vocab_size
操作:
  a. 设置 next_token_id = 256 （下一个可用的 ID）
  b. 对每个 special_token:
     - 检查是否还有空间: if next_token_id >= vocab_size: 停止添加
     - 将特殊标记转为字节: token_bytes = special_token.encode('utf-8')
     - 添加到词表: vocab[next_token_id] = token_bytes
     - 递增 ID: next_token_id += 1
  
输出: 更新后的 vocab, next_token_id

例子:
  如果 special_tokens = ["<|endoftext|>", "<|padding|>"]
  vocab[256] = b'<|endoftext|>'
  vocab[257] = b'<|padding|>'
  next_token_id = 258
```

#### 3.3 初始化合并列表
```
输入: 无
操作:
  - 创建空列表: merges = []
  
输出: merges (空列表)
```

### 第四步：主训练循环（核心）

#### 4.1 统计所有相邻对的频次
```
输入: word_freq (当前的 token 序列字典)
操作:
  a. 创建 pair_counts = Counter() 或空字典
  b. 对 word_freq 中的每个 (token_tuple, freq):
     - 如果 len(token_tuple) < 2: 跳过（无相邻对）
     - 否则，遍历 i 从 0 到 len(token_tuple)-2:
       * 提取相邻对: pair = (token_tuple[i], token_tuple[i+1])
       * 累加频次: pair_counts[pair] += freq
  
输出: pair_counts (字典: (左ID, 右ID) → 总出现次数)

例子:
  token_tuple = (72, 101, 108, 108, 111), freq = 3
  相邻对: (72,101), (101,108), (108,108), (108,111)
  每个对的计数 += 3
```

#### 4.2 选择最高频的相邻对
```
输入: pair_counts
操作:
  a. 如果 pair_counts 为空:
     - 训练结束，退出循环
  
  b. 找最大频次的 pair:
     - best_pair = max(pair_counts, key=pair_counts.get)
     - 或手动遍历找最大值
  
  c. 提取左右 ID:
     - left_id, right_id = best_pair
  
输出: best_pair (最高频的 (左ID, 右ID) 元组)

例子: 
  如果 (108, 108) 出现 50 次最多
  则 best_pair = (108, 108), left_id = 108, right_id = 108
```

#### 4.3 创建新 token 并更新词表
```
输入: best_pair, vocab, next_token_id
操作:
  a. 从词表获取左右字节串:
     - left_bytes = vocab[left_id]
     - right_bytes = vocab[right_id]
  
  b. 合并字节串:
     - new_token_bytes = left_bytes + right_bytes
  
  c. 记录合并规则:
     - merges.append((left_bytes, right_bytes))
  
  d. 添加新 token 到词表:
     - vocab[next_token_id] = new_token_bytes
     - new_token_id = next_token_id （记住这个 ID 用于替换）
  
输出: 更新的 merges, vocab, new_token_id

例子:
  如果 left_id=108 (b'l'), right_id=108 (b'l')
  则 new_token_bytes = b'll'
  merges 添加 (b'l', b'l')
  vocab[258] = b'll' （假设 next_token_id=258）
```

#### 4.4 执行全量替换（左到右、不重叠）
```
输入: word_freq, best_pair, new_token_id
操作:
  a. 创建新字典 updated_word_freq = {}
  
  b. 对每个 (token_tuple, freq) in word_freq:
     执行替换算法:
     1. 创建结果列表 result = []
     2. 设置索引 i = 0
     3. while i < len(token_tuple):
        - 如果 i+1 < len(token_tuple) 且 
          (token_tuple[i], token_tuple[i+1]) == best_pair:
          * result.append(new_token_id)
          * i += 2  （跳过这两个已合并的 token）
        - 否则:
          * result.append(token_tuple[i])
          * i += 1
     
     4. 将结果转为元组: merged_tuple = tuple(result)
     5. 累加频次: updated_word_freq[merged_tuple] += freq
  
  c. 替换原字典: word_freq = updated_word_freq
  
输出: 更新后的 word_freq

详细例子:
  原始: (72, 101, 108, 108, 111) = (H, e, l, l, o)
  best_pair = (108, 108) = (l, l)
  new_token_id = 258
  
  替换过程:
  i=0: (72,101) != (108,108), 添加 72, i=1
  i=1: (101,108) != (108,108), 添加 101, i=2
  i=2: (108,108) == (108,108), 添加 258, i=4
  i=4: 添加 111, i=5
  
  结果: (72, 101, 258, 111) = (H, e, ll, o)
```

#### 4.5 更新循环控制变量
```
输入: next_token_id, vocab_size
操作:
  a. 递增 token ID:
     - next_token_id += 1
  
  b. 检查终止条件:
     - 如果 next_token_id >= vocab_size:
       * 词表已满，退出循环
     - 否则:
       * 继续下一轮（回到 4.1）
  
输出: 更新的 next_token_id，或终止信号
```

### 第五步：返回结果
```
输入: vocab, merges
操作:
  - 直接返回元组: return (vocab, merges)
  
输出: 
  - vocab: 完整的词表字典 {token_id: bytes}
  - merges: 按顺序的合并规则列表 [(left_bytes, right_bytes), ...]
```

## 完整流程示例

让我们通过一个小例子走完整个流程：

### 输入数据
```
文本: "aa bb aa"
vocab_size: 258
special_tokens: []
```

### 执行过程

**初始状态:**
- 预分词: ["aa", " ", "bb", " ", "aa"]
- 统计频次: {"aa": 2, " ": 2, "bb": 1}
- 转为字节: 
  - "aa" → (97, 97): 频次 2
  - " " → (32,): 频次 2
  - "bb" → (98, 98): 频次 1
- vocab: {0: b'\x00', ..., 97: b'a', 98: b'b', ..., 255: b'\xff'}
- merges: []

**第一轮:**
1. 统计相邻对:
   - (97, 97) 出现 2 次（来自 "aa"）
   - (98, 98) 出现 1 次（来自 "bb"）
2. 最高频: (97, 97)
3. 创建新 token:
   - new_token_bytes = b'a' + b'a' = b'aa'
   - vocab[256] = b'aa'
   - merges = [(b'a', b'a')]
4. 替换:
   - (97, 97) → (256,)
   - word_freq 变为: {(256,): 2, (32,): 2, (98, 98): 1}

**第二轮:**
1. 统计相邻对:
   - (98, 98) 出现 1 次
2. 最高频: (98, 98)
3. 创建新 token:
   - new_token_bytes = b'b' + b'b' = b'bb'
   - vocab[257] = b'bb'
   - merges = [(b'a', b'a'), (b'b', b'b')]
4. 替换:
   - (98, 98) → (257,)
   - word_freq 变为: {(256,): 2, (32,): 2, (257,): 1}

**终止:**
- next_token_id = 258 >= vocab_size = 258
- 返回 vocab 和 merges

## 关键实现细节和注意事项

### 1. 数据结构选择
- **为什么用 tuple[int, ...] 作为键？**
  - tuple 是不可变的，可以作为字典键
  - 整数比较比字节串比较更快
  - 便于索引和切片操作

### 2. 左到右不重叠替换的细节
```python
# 错误示例（会重叠）：
for i in range(len(token_tuple)-1):
    if (token_tuple[i], token_tuple[i+1]) == best_pair:
        # 替换但不跳过，会导致重叠

# 正确示例（不重叠）：
i = 0
while i < len(token_tuple):
    if i+1 < len(token_tuple) and (token_tuple[i], token_tuple[i+1]) == best_pair:
        # 替换并跳过两个位置
        i += 2
    else:
        # 保留当前，移动一个位置
        i += 1
```

### 3. 特殊标记处理要点
- 必须在正则表达式前面，确保优先完整匹配
- 统计频次时必须排除，避免被拆分合并
- 添加到词表时占用 ID 空间，影响 vocab_size 计算

### 4. 性能优化技巧
- 使用 Counter 而不是手动维护字典
- 批量处理而不是逐个处理
- 避免不必要的类型转换（如反复 bytes ↔ tuple）
- 合并后的 word_freq 可以用 defaultdict(int) 避免检查键存在

### 5. 边界情况处理
- 空文件：直接返回只有 256 个字节的初始词表
- vocab_size < 256：逻辑错误，应报错或早停
- 所有 token 长度为 1：无相邻对，提前结束
- 特殊标记超过剩余空间：只添加能放下的部分

## 背景与参数为何重要

### 为什么做“字节级”BPE
- 通用性：任何 Unicode 文本都能被 UTF-8 编码成 0–255 的字节序列，初始 256 个字节 token 能覆盖所有输入，无 OOV（out-of-vocabulary）。
- 可逆性与稳定性：在字节层面合并与还原是逐字节确定的，不依赖语言学规则，跨语言一致。
- 跨域适配：不同语种/符号（含 emoji）无需为字符集特判；常见字节序列会被合并成更长 token，提高压缩率。
- 权衡：非 ASCII 字符一个字符通常占多字节，初始会变长，但 BPE 通过统计常见序列（如中文/emoji 的多字节组合）进行合并，逐步缩短。

### 为什么需要预分词（GPT-2 正则）
- 约束合并范围：先把文本切成更大的“词/符号/空白”单元，避免跨词/跨空白的合并，使学习到的 merges 更贴近自然语言结构。
- 一致性：测试与业界实现（如 GPT-2/tiktoken）使用相同的正则，确保结果可比、可复现。
- 性能：预分词减少候选 pair 的组合爆炸，统计更高效。
- 特殊 token 保护：把特殊 token 放在正则前面优先匹配，保证它们不被拆分，以免训练把控制标记污染到普通词表里。

### 参数含义与影响
- `input_path`（语料来源）：
  - 决定统计到的高频片段，从而决定 merges 的形状；域变更（英文→代码/中文）会显著改变 merges。
  - 小语料运行更快、合并更“窄”；大语料更全面但耗时，需注意 I/O 与内存（本作业给定小样本）。
  - 需确保 UTF-8 解码一致；非法字节可选择忽略或报错，训练一般用干净语料。
- `vocab_size`（词表上限）：
  - 越大，能合并的 token 越多，序列更短；嵌入矩阵更大、内存占用更高，推理 softmax 也更重。
  - 过小：压缩不足、上下文同长度下有效 token 更少；过大：收益递减且占用资源。
  - 定义包含：256 个初始字节 + 特殊 token + 训练产生的新 token。
- `special_tokens`（特殊标记）：
  - 用于边界/控制（如 `<|bos|>`、`<|eot|>`、系统提示等），必须作为“原子”存在，不能被拆分或参与合并。
  - 仅加入词表，不参与频次统计与 merges；这样能够在下游稳定地插入/识别这些控制符。

### 为什么 merges 是“有序的字节对列表”
- BPE 的应用阶段依赖合并顺序：早先产生的合并拥有更高优先级。通常会将 merges 建立为“对→秩”的映射以贪心应用。
- 返回字节对（而不是 id 对）可移植性更强：只要有 `vocab` 与 `merges` 就能在别处重建 tokenizer。
- 顺序决定了分词的确定性：同样文本在同样 merges 下会得到一致的分词结果。

### 左到右、不重叠替换的原因
- 这是原始 BPE 算法的关键约束：在某一轮，只允许把“本轮选中的 pair”各自合并一次，且不与自身重叠，防止一轮内重复放大或改变统计基准。
- 这样能确保每一轮的计数与替换是对齐的，训练具有可解释的“逐步构造”特性。

### 为什么使用 `regex` 包与 GPT-2 正则
- 功能：支持 Unicode 类别（\p{L}/\p{N} 等）与负向前瞻，是 GPT-2 预分词模式的必要特性；标准库 `re` 可能不支持或性能不足。
- 性能：在本作业提供的小语料上足够快，也与社区实现的表现一致。

### 初始 256 字节映射的动机
- 8 位字节天然有 256 个取值；UTF-8 任意字符都能分解到这 256 个基础单元上，从而保证任何输入都可表示、可还原。
- GPT-2 的 `bytes_to_unicode` 仅用于把不可打印字节映射到可打印字符，便于序列化与调试，不影响 BPE 在字节层面的合并。

### 性能门槛与复杂度直觉
- 每轮：统计相邻对 O(L)，全量替换 O(L)；总计 O(R·L)。在测试用小语料下，朴素实现即可达标（< 1.5s）。
- 常数优化：尽量避免在循环中做无谓的复制与转换，使用 `Counter` 与不可变 `tuple` 做键。

### 可复现性
- 给定固定语料、参数与实现细节（尤其是预分词与替换规则），训练输出的 `merges` 顺序是确定的；这对下游对齐与测试通过至关重要。
